# 0. 데이터 베이스 만들기

데이터 출처: https://www.kaggle.com/datasets/bharatmaheshwarids/5000-employee-dataset-of-an-organization-dummy

데이터 컬럼:
1. Name(이름)
2. Address(주소)
3. Salary(임금)
4. DOJ(입사 날)
5. DOB(탄생일)
6. Age(나이)
7. Sex(성별)
8. Dependents(부양가족의 수)
9. HRA(기본 급여에 대해 받는 주택임대 수당)
10. DA(기본 급여에 대해 받는 물가수당)
11. PF(급여에서 공제되는 직원 공적 기금(연금))
12. Gross Salary(세금 공제 전 총 급여)
13. Insurance(직원이 가지고 있는 보험)
14. Marital Status(결혼 상태)
15. In Company Years(이 회사에서 일한 년수)
16. Year of Experience(
17. Department(일하는 부서)
18. Position(직무)

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine

In [3]:
df= pd.read_csv("Dummy_5000_Employee_Details_Dataset.csv", encoding='utf-8')

In [4]:
engine = create_engine("sqlite:///employee.db")

In [5]:
df.to_sql("employee", engine, index=False)

5000

# 1. 단계별로 진행하기

In [21]:
import os
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from sqlalchemy import text
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate

In [8]:
db = SQLDatabase.from_uri("sqlite:///employee.db")
print(db.get_usable_table_names())

['employee']


In [9]:
db.run("SELECT * FROM employee LIMIT 1")

"[(0, 0, 'VgwbPyBn', 'jyNxKZtk St, piZQhYcARo, PUN 172813', 149289.04, '1998-04-27', '1971-03-27', 53, 'Other', 2.0, 8776.095999999998, 35233.66637469586, 22142.724764963503, 171156.07760973237, None, 'Widowed', 26, 32, 'Sales', 'Sales Director')]"

In [17]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)
sql = chain.invoke({"question": "회사에서 결혼 안한 직원들은 몇명이야?"})
sql

'SELECT COUNT("index") \nFROM employee \nWHERE "Marital Status" = \'Single\' OR "Marital Status" = \'Divorced\' OR "Marital Status" = \'Widowed\''

In [26]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [16]:
response = db.run(sql)
print(response)

[(1948,)]


In [18]:
execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "회사에서 결혼 안한 직원들은 몇명이야?"})

'[(1948,)]'

In [25]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer: """
)

answer_chain=answer_prompt|llm
question="회사에서 결혼 안한 직원들은 몇명이야?"
answer=answer_chain.invoke({"question": question, "query": sql, "result": response})
print(answer.content)

결혼을 하지 않은 직원은 1948명입니다.


In [ ]:
#한번에 해보고 싶을때!
all_chain=

## 2. agent 사용하기

In [14]:
from langchain_community.agent_toolkits import create_sql_agent
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
agent_executor.invoke({"input": "회사에서 결혼 안한 직원들은 몇명이야?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


employee
Invoking: `sql_db_schema` with `{'table_names': 'employee'}`



CREATE TABLE employee (
	"index" BIGINT, 
	"Unnamed: 0" BIGINT, 
	"Name" TEXT, 
	"Address" TEXT, 
	"Salary" FLOAT, 
	"DOJ" TEXT, 
	"DOB" TEXT, 
	"Age" BIGINT, 
	"Sex" TEXT, 
	"Dependents" FLOAT, 
	"HRA" FLOAT, 
	"DA" FLOAT, 
	"PF" FLOAT, 
	"Gross Salary" FLOAT, 
	"Insurance" TEXT, 
	"Marital Status" TEXT, 
	"In Company Years" BIGINT, 
	"Year of Experience" BIGINT, 
	"Department" TEXT, 
	"Position" TEXT
)

/*
3 rows from employee table:
index	Unnamed: 0	Name	Address	Salary	DOJ	DOB	Age	Sex	Dependents	HRA	DA	PF	Gross Salary	Insurance	Marital Status	In Company Years	Year of Experience	Department	Position
0	0	VgwbPyBn	jyNxKZtk St, piZQhYcARo, PUN 172813	149289.04	1998-04-27	1971-03-27	53	Other	2.0	8776.095999999998	35233.66637469586	22142.724764963503	171156.07760973237	None	Widowed	26	32	Sales	Sales Director
1	1	pgdYaIWk	PypVsqug 

{'input': '회사에서 결혼 안한 직원들은 몇명이야?', 'output': '회사에서 결혼 안한 직원은 1,948명입니다.'}